In [1]:
import pandas as pd
import bamboolib as bam

In [2]:
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'


In [3]:
confirmed = pd.read_csv(url_confirmed)
deaths = pd.read_csv(url_deaths)
recovered = pd.read_csv(url_recovered)

In [4]:
# Unpivot data frames
date1 = confirmed.columns[4:]
total_confirmed = confirmed.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=date1, var_name='date', value_name='confirmed')
date2 = deaths.columns[4:]
total_deaths = deaths.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=date2, var_name='date', value_name='death')
date3 = recovered.columns[4:]
total_recovered = recovered.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=date3, var_name='date', value_name='recovered')


In [5]:
#merging dataframes
covid_data = total_confirmed.merge(right = total_deaths, how = 'left', on = ['Province/State', 'Country/Region', 'date', 'Lat', 'Long'])
covid_data = covid_data.merge(right = total_recovered, how = 'left', on = ['Province/State', 'Country/Region', 'date', 'Lat', 'Long'])

In [6]:
# Converting date column from string to proper date format
covid_data['date'] = pd.to_datetime(covid_data['date'])


In [7]:
covid_data = covid_data.drop(columns=['Province/State', 'recovered'])

In [8]:
covid_data_1 = covid_data.groupby(['date'])[['confirmed', 'death',]].sum().reset_index()

In [9]:
covid_data_africa = covid_data.loc[covid_data['Country/Region'].isin(['Algeria', 'Angola', 'Benin', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', "Cote d\\'Ivoire", 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Liberia', 'Libya', 'Lesotho', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Zambia', 'Zimbabwe'])]


In [10]:
covid_data_afr_1 = covid_data_africa.groupby(['date'])[['confirmed', 'death']].sum().reset_index()

In [11]:
covid_data_afr_1

          date  confirmed   death
0   2020-01-22          0       0
1   2020-01-23          0       0
2   2020-01-24          0       0
3   2020-01-25          0       0
4   2020-01-26          0       0
..         ...        ...     ...
990 2022-10-08   11777434  249548
991 2022-10-09   11777996  249548
992 2022-10-10   11778951  249550
993 2022-10-11   11780192  249553
994 2022-10-12   11780887  249554

[995 rows x 3 columns]

In [12]:
#Make a dict of Locations

covid_data_list = covid_data[['Country/Region', 'Lat', 'Long']]
dict_of_locations = covid_data_list.set_index('Country/Region')[['Lat', 'Long']].T.to_dict('dict')

In [13]:
#african dict of locations
covid_data_africa_list = covid_data_africa[['Country/Region', 'Lat', 'Long']]
dict_of_locations_afr = covid_data_africa_list.set_index('Country/Region')[['Lat','Long']].T.to_dict('dict') 

In [14]:
top_ten_confirmed = covid_data_africa.groupby('Country/Region')['confirmed'].max().sort_values(ascending=False).reset_index().head(10)

In [15]:
top_ten_confirmed

  Country/Region  confirmed
0   South Africa    4022107
1        Morocco    1265138
2        Tunisia    1145930
3          Egypt     515645
4          Libya     507012
5       Ethiopia     493723
6          Kenya     338528
7         Zambia     333624
8        Algeria     270722
9        Nigeria     265816

In [16]:
top_ten_death = covid_data_africa.groupby('Country/Region')['death'].max().sort_values(ascending = False).reset_index().head(10)

In [17]:
top_ten_death

  Country/Region   death
0   South Africa  102194
1        Tunisia   29254
2          Egypt   24797
3        Morocco   16278
4       Ethiopia    7572
5        Algeria    6881
6          Libya    6437
7          Kenya    5678
8       Zimbabwe    5604
9          Sudan    4963

In [28]:
import plotly.graph_objs as go

fig = go.Figure(data=[go.Bar(x=top_ten_death['Country/Region'], 
                             y=top_ten_death['death'], 
                            marker = dict(color = 'orange'),
                            hoverinfo='text',
                             hovertext=
                             '<b>Death</b>: ' +top_ten_death['death'].astype(str) + '<br>' +
                             '<b>Country</b>: ' + top_ten_death['Country/Region'].astype(str) + '<br>')] 
               )

fig.update_layout(title = {'text': 'test',
                           'y': 0.93,
                           'x': 0.5,
                           'xanchor': 'center',
                           'yanchor': 'top'},
                  paper_bgcolor='#1f2c56',
                  plot_bgcolor='#1f2c56',
                  
                          )